# Descarga de submissions de Reddit mediante el modulo Pushshift usando Python

Para descarga de información en Reddit contamos con muchas herramientas como PRAW, PMAW y requests.

PMAW es derivado de PRAW y ambos tienen algunas limitantes, como la eliminación del   *subreddit.submissions* que nos permite seleccionar la información que será descargada.

Es por esto que se opta por utilizar  *requests* para acceder a Reddit y obtener la información almacenada en **Pushshift**.



# Importar módulos

In [ ]:
import pandas as pd # Uso de dataframes
import requests # Método para acceder a Pushshift por mediante url 
import json # Manipulación de información JSON
import csv # Para convertir las tablas finales en archivos csv y guardarla en la máquina local
import time # Convertir el tiempo UTC en tiempo GMT
import datetime # Manipulación del tiempo

# Ejemplos de URLS de Pushshift

In [ ]:
# Se puede acceder al API de Pushshift a través de una URL con los parámetros relevantes sin necesidad de identificarse en Reddit
# Estos son algunos ejemplos de URLs que generan una página con información JSON
busca_gaming_despues_fecha = "https://api.pushshift.io/reddit/search/submission/?q=screenshot&after=1514764800&before=1517443200&subreddit=gaming"
busca_science = "https://api.pushshift.io/reddit/search/submission/?q=science"

# Parámetros para el URL de Pushshift
Estos son los parámetros que se conideran más importantes para la consulta de Reddit, utilizados para la construcción del URL de Pushshift:

* size — Tamaño de entradas devueltas, modificado a 1,000
* after — Dónde comienza la búsqueda 
* before — Dónde finaliza la búsqueda
* title — Busca submissions que contengan la palabra en el título
* subreddit — Busca en un Subreddit en particular

Todo esto con la finalidad de reducir el tamaño de la búsqueda.

In [ ]:
# Adaptado de https://gist.github.com/dylankilkenny/3dbf6123527260165f8c5c3bc3ee331b
# Se construye la URL de Pushshift, accediendo a una pagina web donde está almacenada la información en JSON a manera de lista
def getPushshiftData(query, after, before, sub):
    # Construye la URL de Pushshift con los parámetros antes mencionados
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    # Muestra al usuario la URL 
    print(url)
    # Solicitud de la URL
    r = requests.get(url)
    r.raise_for_status()
    # Carga la pagina con la información JSON en una variable
    if (r.status_code != 204):
      data = json.loads(r.text)
    #regresa la información de la variable donde está contenida
    return data['data']

# Extrae la información clave de Submissions

Es aquí donde tenemos la libertad de solicitar la información que requerimos de Submissions para nuestro análisis: 
* Title: Título de la noticia
* URL: URL de la noticia
* Subreddit: Grupo en el que se publicó el submission
* Selftext: Contenido de la noticia, no todas las noticiac cuentan con esto
* Flair: Tópico del submission
* Author: Usuario que publicó el submission
* ID: ID para identificar el submission
* Score: Puntuación del submission
* Created_utc: Fecha de publicación
* Num_comments: Número de comentarios 
* Permalink: Enlace permanente.


In [ ]:
# Extracción de la información específica de los datos JSON generados  
def collectSubData(subm):
    #subData fue creado al inicio donde está toda la información para ser agregados a la variable global subStats.
    subData = list() # lista para almacenar la información
    title = subm['title'] # Titulo 
    url = subm['url'] # URL
    subreddit = subm['subreddit'] # Grupo Subreddit
    # selftext, o cuerpo del post, no siempre está presente en submissions, para evitar errores se usa try/except
    try:
      selftext = subm['selftext']
    except KeyError:
      selftext = "NaN"
    #flairs no siempre está presente en submissions, para evitar errores se usa try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author'] # Autor del post
    sub_id = subm['id'] # ID del post
    score = subm['score'] # Puntuación del post
    created = datetime.datetime.fromtimestamp(subm['created_utc']) # Fecha de creación en UTC, por lo que se hace la conversión
    numComms = subm['num_comments'] # Número de comentarios
    permalink = subm['permalink'] # Linl permanente

    #Junta toda la información en una tubla y se agrega a subData
    subData.append((sub_id,title,url,subreddit,selftext,author,score,created,numComms,permalink,flair))
    # Crea un diccionario con la entrada de información del submission actual y almacena toda la información relacionada con el submission
    subStats[sub_id] = subData

# Update your Search Settings here

In [ ]:
# Para tener el formato correcto de tiempo de la URL se puede usar la siguiente página
#https://www.unixtimestamp.com/index.php > Esto para crear el propio timestamp
after = "1608310800" #Submissions despues del timestamp
before = "1609459200" #Submissions antes del timestamp
query = "GME" # Palabra clave para buscar en submissions
sub = "wallstreetbets" #Cual Subreddit se buscará la información

#subCount cuenta el no. del total de envíos que se recopila
subCount = 0
#subStats es el diccionario donde almacenaremos nuestros datos.
subStats = {}

In [ ]:
# Necesitamos ejecutar esta función fuera del ciclo primero para obtener después la variable actualizada
data = getPushshiftData(query, after, before, sub)
# Se ejecutará hasta que se hayan recopilado todas las publicaciones, es decir, cuando la longitud de la variable de datos = 0
# desde la fecha 'posterior' hasta la fecha 'anterior'
while len(data) > 0: #La longitud de los datos es el número de submission(data[0],data[1], etc.), una vez que llega a cero (después y antes de que vars sean iguales) finaliza
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Llama getPushshiftData() Con la fecha creada del ultimo submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #actualiza la variable 'after' a la última fecha creada del submission
    after = data[-1]['created_utc']
    # datos han cambiado debido a la nueva variable posterior proporcionada por el código anterior
    data = getPushshiftData(query, after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1608310800&before=1609459200&subreddit=wallstreetbets
100
2020-12-18 17:59:27
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1608314367&before=1609459200&subreddit=wallstreetbets
100
2020-12-18 19:27:11
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1608319631&before=1609459200&subreddit=wallstreetbets
97
2020-12-18 20:43:50
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1608324230&before=1609459200&subreddit=wallstreetbets
100
2020-12-18 21:38:27
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1608327507&before=1609459200&subreddit=wallstreetbets
100
2020-12-18 22:57:53
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1608332273&before=1609459200&subreddit=wallstreetbets
100
2020-12-19 00:54:41
https://api.pushshift.io/reddit/search/submission/?title=&size=1000&after=1608339281&befo

# Revisión de que la extracción del submission se haya cumplido con exito

In [ ]:
print(str(len(subStats)) + " submissions agregado a la lista")
print("1st entrada es:")
print(list(subStats.values())[0][0][1] + " creado por: " + str(list(subStats.values())[0][0][5]))
print("Ultima entrada es:")
print(list(subStats.values())[-1][0][1] + " creado por: " + str(list(subStats.values())[-1][0][5]))

8098 submissions have added to list
1st entry is:
rock solid right now😍😍 created: B19B0y124
Last entry is:
Why is no one talking about RKT of we're trying to go to the moon? created: NWMixOlogist


# Guardado de información en archivo CSV

In [ ]:
def updateSubs_file():
    upload_count = 0
    print("Ingresa el nombre del archivo agregando .csv al final ")
    filename = input() #Nombre que puso al archivo csv
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Subreddit","Body","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions han sido guardados")
updateSubs_file()

input filename of submission file, please add .csv
wallstreetbets_2020_12.csv
12432 submissions have been uploaded
